In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import sys
sys.path.append('../models')
from vgg import VGG16_model

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### Load the Pocovid Dataset

In [3]:
import sys
sys.path.append('../scripts')
from pocovid_dataset import PocovidDataset
from torch.utils.data import Dataset, DataLoader

In [4]:
train_pct = 0.8
batch_size = 1
num_workers = 2

dataset = PocovidDataset(root_dir='../data/image_dataset',
                                     transform=transforms.Compose([
                                       transforms.Resize((224,224)),
                                       transforms.RandomAffine(10,translate=(0.1,0.1)),
                                       transforms.ToTensor()
                                       ]))

dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=num_workers)

train_len = int(len(dataloader) * train_pct)

trainset, valset = torch.utils.data.random_split(dataset, [train_len, len(dataset) - train_len])

train_loader = torch.utils.data.DataLoader(trainset, num_workers=num_workers, shuffle=True,
                                  batch_size=batch_size)

val_loader = torch.utils.data.DataLoader(valset, num_workers=num_workers, shuffle=False,
                                batch_size=batch_size)

In [5]:
len(dataset)

1137

In [6]:
class_map = {'covid' : dataset.get_covid_class_idx(),
            'pneumonia' : dataset.get_pneu_class_idx(),
            'regular' : dataset.get_regular_class_idx()}

In [7]:
train_loader

In [8]:
for data in (train_loader):
    print(data)
    break

[tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0706, 0.0706, 0.0667,  ..., 0.0000, 0.0000, 0.0000],
          [0.0706, 0.0745, 0.0784,  ..., 0.0000, 0.0000, 0.0000],
          [0.0784, 0.0863, 0.0902,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0706, 0.0706, 0.0667,  ..., 0.0000, 0.0000, 0.0000],
          [0.0706, 0.0745, 0.0784,  ..., 0.0000, 0.0000, 0.0000],
          [0.0784, 0.0863, 0.0902,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 

### Model

In [9]:
model = VGG16_model()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/nathan/.cache/torch/hub/checkpoints/vgg16-397923af.pth
4.4%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

10.6%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

16.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_

In [12]:
model = model.cuda()

### Loss function and Optimizer

In [13]:
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(params = model.parameters(), lr=1e-4)

### Training Loop

In [14]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        print(i, data)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

0 [tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0471, 0.0471, 0.0471,  ..., 0.0000, 0.0000, 0.0000],
          [0.0471, 0.0471, 0.0510,  ..., 0.0000, 0.0000, 0.0000],
          [0.0471, 0.0471, 0.0471,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0471, 0.0471, 0.0471,  ..., 0.0000, 0.0000, 0.0000],
          [0.0471, 0.0471, 0.0510,  ..., 0.0000, 0.0000, 0.0000],
          [0.0471, 0.0471, 0.0471,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ...

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 64])